## Made my own csv of the html table from http://www.thegreenpapers.com/G12/HouseVoteByParty.phtml

### In the Candidate column (row index 2): * = winner, (I) = incumbent

### Resorted in excel by state and then district because some districts were out of order and my code doesn't account for that

In [ ]:
# import csv
import json
# import numpy as np
import pandas as pd

In [ ]:
#dictionary between FIPS and states
with open("../data/st2Fips.json","rb") as f3:
    fips = json.load(f3)

# read into pandas data frame
df = pd.read_csv('../data/113_2012_house_popular_vote.csv', header=0, usecols = [0,1,2,3,4], names = ['state','cd','name','party','numVotes'])

df = df[:len(df)-1]

nRows = len(df)

# remove leading and trailing whitespace
df["state"] = df["state"].map(str.strip)
df["cd"] = df["cd"].map(str.strip)
df["name"] = df["name"].map(str.strip)
df["party"] = df["party"].map(str.strip)

# set "At Large" districts to 00
df['cd'][df['cd'] == 'At-Large'] = '00'

df['cd'] = df['cd'].str.replace('CD ','')

# convert votes to numbers
df['numVotes'] = df['numVotes'].str.replace(',','').convert_objects(convert_numeric=True)


In [ ]:
thisStateAbbrev = ''
thisDistrict = ''

c113_dict = {}

for i in range(len(df)):
    if df['state'][i] != thisStateAbbrev:
        thisStateAbbrev = df['state'][i]
        # found a new state, start a new temporary dictionary
        stateDict = {}
        stateDict['state'] = thisStateAbbrev
        # initialize district party tally
        stateDict['demDistr'] = 0
        stateDict['repDistr'] = 0
        stateDict['othDistr'] = 0
        # initialize vote tally
        stateDict['demVotes'] = 0
        stateDict['repVotes'] = 0
        stateDict['othVotes'] = 0
    
    if thisStateAbbrev in fips:
        cd = df['cd'][i]

        newDistrict = False
        if cd != thisDistrict:
            thisDistrict = cd
            newDistrict = True
            # found a new district, start a new temporary dictionary
            cdDict = {}

            # initialize vote tally
            cdDict['repVotes'] = 0
            cdDict['demVotes'] = 0
            cdDict['othVotes'] = 0

        candidate_name = df['name'][i].decode("ascii", "ignore")
        if candidate_name[0] == '*':
            if 'Democratic' in df['party'][i]:
                thisParty = 'D'
                stateDict['demDistr'] += 1
            elif 'Republican' in df['party'][i]:
                thisParty = 'R'
                stateDict['repDistr'] += 1
            else:
                thisParty = 'O'
                stateDict['othDistr'] += 1
            # found the winner for this district
            # if candidate_name[-3:] == '(I)':
            #     cdDict['incumbent'] = True
            # else:
            #     cdDict['incumbent'] = False
            cdDict['name'] = candidate_name.replace('*','').replace(' (I)','')
            cdDict['party'] = thisParty

        if newDistrict or cd == thisDistrict:
            # tally dem, rep, and other votes
            if pd.isnull(df['numVotes'][i]) == False:
                nVotes = df['numVotes'][i]
                if 'Democratic' in df['party'][i]:
                    partyV = 'demVotes'
                elif 'Republican' in df['party'][i]:
                    partyV = 'repVotes'
                else:
                    partyV = 'othVotes'

                cdDict[partyV] += nVotes
                stateDict[partyV] += nVotes
            
            if ((i+1) < len(df) and thisDistrict != df['cd'][i+1]) or i+1 == len(df):
                stateDict[cd] = cdDict
        
        if ((i+1) < len(df) and thisStateAbbrev != df['state'][i+1]) or i+1 == len(df):
            c113_dict[fips[thisStateAbbrev]] = stateDict


In [ ]:
with open('../data/113_2012_house_popular_vote.json', 'wb') as f:
    f.write(json.dumps(c113_dict))